In [2]:
# import packages

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Job Placement Data Set Question: Can we predict which individuals will be placed in jobs based on information about the student?

The business metric for this problem would be to increase the amount of students (customers) at the XYZ campus.

In [3]:
job_placement = pd.read_csv("https://raw.githubusercontent.com/DG1606/CMS-R-2020/master/Placement_Data_Full_Class.csv")

In [4]:
job_placement.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [5]:
job_placement.dtypes


sl_no               int64
gender             object
ssc_p             float64
ssc_b              object
hsc_p             float64
hsc_b              object
hsc_s              object
degree_p          float64
degree_t           object
workex             object
etest_p           float64
specialisation     object
mba_p             float64
status             object
salary            float64
dtype: object

In [6]:
object_list = list(job_placement.select_dtypes('object')) #select function to find the categorical variables and create a list  

job_1h = pd.get_dummies(job_placement, columns = object_list) 

job_1h

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary,gender_F,gender_M,ssc_b_Central,...,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,workex_No,workex_Yes,specialisation_Mkt&Fin,specialisation_Mkt&HR,status_Not Placed,status_Placed
0,1,67.00,91.00,58.00,55.0,58.80,270000.0,False,True,False,...,False,False,False,True,True,False,False,True,False,True
1,2,79.33,78.33,77.48,86.5,66.28,200000.0,False,True,True,...,True,False,False,True,False,True,True,False,False,True
2,3,65.00,68.00,64.00,75.0,57.80,250000.0,False,True,True,...,False,True,False,False,True,False,True,False,False,True
3,4,56.00,52.00,52.00,66.0,59.43,NaN,False,True,True,...,True,False,False,True,True,False,False,True,True,False
4,5,85.80,73.60,73.30,96.8,55.50,425000.0,False,True,True,...,False,True,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,80.60,82.00,77.60,91.0,74.49,400000.0,False,True,False,...,False,True,False,False,True,False,True,False,False,True
211,212,58.00,60.00,72.00,74.0,53.62,275000.0,False,True,False,...,True,False,False,True,True,False,True,False,False,True
212,213,67.00,67.00,73.00,59.0,69.72,295000.0,False,True,False,...,False,True,False,False,False,True,True,False,False,True
213,214,74.00,66.00,58.00,70.0,60.23,204000.0,True,False,False,...,False,True,False,False,True,False,False,True,False,True


In [7]:
prevalence = job_1h.status_Placed.value_counts()[True]/len(job_1h.status_Placed)
prevalence

np.float64(0.6883720930232559)

In [8]:
print(job_1h.status_Placed.value_counts())

status_Placed
True     148
False     67
Name: count, dtype: int64


In [9]:
job_dt = job_1h.drop(['salary', 'sl_no'], axis=1)
job_dt

,ssc_p,hsc_p,degree_p,etest_p,mba_p,gender_F,gender_M,ssc_b_Central,ssc_b_Others,hsc_b_Central,...,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,workex_No,workex_Yes,specialisation_Mkt&Fin,specialisation_Mkt&HR,status_Not Placed,status_Placed
0,67.00,91.00,58.00,55.0,58.80,False,True,False,True,False,...,False,False,False,True,True,False,False,True,False,True
1,79.33,78.33,77.48,86.5,66.28,False,True,True,False,False,...,True,False,False,True,False,True,True,False,False,True
2,65.00,68.00,64.00,75.0,57.80,False,True,True,False,True,...,False,True,False,False,True,False,True,False,False,True
3,56.00,52.00,52.00,66.0,59.43,False,True,True,False,True,...,True,False,False,True,True,False,False,True,True,False
4,85.80,73.60,73.30,96.8,55.50,False,True,True,False,True,...,False,True,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,80.60,82.00,77.60,91.0,74.49,False,True,False,True,False,...,False,True,False,False,True,False,True,False,False,True
211,58.00,60.00,72.00,74.0,53.62,False,True,False,True,False,...,True,False,False,True,True,False,True,False,False,True
212,67.00,67.00,73.00,59.0,69.72,False,True,False,True,False,...,False,True,False,False,False,True,True,False,False,True
213,74.00,66.00,58.00,70.0,60.23,True,False,False,True,False,...,False,True,False,False,True,False,False,True,False,True


In [10]:
Train, Test = train_test_split(job_dt, train_size=58, stratify=job_dt.status_Placed)
print(Train.shape)
print(Test.shape)

(58, 23)
(157, 23)


In [11]:
Tune, Test = train_test_split(Test,  train_size = 58)

print(Train.status_Placed.value_counts())

status_Placed
True     40
False    18
Name: count, dtype: int64


In [12]:
print(Tune.status_Placed.value_counts())

status_Placed
True     37
False    21
Name: count, dtype: int64


In [13]:
print(Test.status_Placed.value_counts())

status_Placed
True     71
False    28
Name: count, dtype: int64


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score

# Initialize the DecisionTreeClassifier
dtree = DecisionTreeClassifier()

# separate features and target variable
X_train = Train.drop('status_Placed', axis=1)
y_train = Train['status_Placed']
X_test = Test.drop('status_Placed', axis=1)
y_test = Test['status_Placed']

# Fit the model on the training data
dtree.fit(X_train, y_train)

# Predict on the test data
y_pred_dtree = dtree.predict(X_test)

# Calculate precision
precision = precision_score(y_test, y_pred_dtree)


In [15]:
def stu_placement(x):
    object_list = list(job_placement.select_dtypes('object'))

    job_1h = pd.get_dummies(job_placement, columns = object_list) 
    prevalence = job_1h.status_Placed.value_counts()[True]/len(job_1h.status_Placed)
    job_dt = job_1h.drop(['salary', 'sl_no'], axis=1)
    Train, Test = train_test_split(job_dt, train_size=58, stratify=job_dt.status_Placed)
    Tune, Test = train_test_split(Test,  train_size = 58)
    dtree = DecisionTreeClassifier()
    
    X_train = Train.drop('status_Placed', axis=1)
    y_train = Train['status_Placed']
    _test = Test.drop('status_Placed', axis=1)
    y_test = Test['status_Placed']

    dtree.fit(X_train, y_train)

    y_pred_dtree = dtree.predict(X_test)
    precision = precision_score(y_test, y_pred_dtree)

    return(f'Precision: {precision}')



In [16]:
stu_placement(job_placement)

'Precision: 0.6056338028169014'

### Step 3:
From the precision rate provided, I believe that this model is working well, however it could be better as around 30% of the positives identified by the model are false positives. In this context, it means that around 30% of the time the model predicted that the student would be placed, however they were not. One thing that worries me about this model is that if it were put to use in the real world, it could be very harmful to students who are given this false positive. This model would have to be taken lightly, and while it may help address our problem to some extent, people would have to be informed that the model can be inaccurate. 

## College Completion Data Set Question: Based on the aid percentile that the school is in, can we predict the retention rate?

The independent business metric for this problem would be the increase in revenue for the University.

In [20]:
college_completion = pd.read_csv("/workspaces/DS-3021/data/cc_institution_details.csv")
college_completion.retain_percentile.value_counts()

retain_percentile
91.0     64
0.0      59
8.0      53
38.0     52
62.0     47
         ..
50.0     27
65.0     26
36.0     25
53.0     25
100.0    10
Name: count, Length: 101, dtype: int64

In [18]:
college_completion.head()

,index,unitid,chronname,city,state,level,control,basic,hbcu,flagship,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [ ]:
college_completion['retention_class'] = college_completion['retain_percentile'].apply(lambda x: 1 if x >= 50 else 0)

to_drop = list(range(39, 56))
grad_data1 = college_completion.drop(college_completion.columns[to_drop], axis=1)
drop_more = [0,2,3,6,8,11,12,14,15,18,21,23,29,32,33,34,35]
grad_data2 = grad_data1.drop(grad_data1.columns[drop_more], axis=1)
grad_data2.head()

,unitid,state,level,basic,flagship,long_x,student_count,awards_per_natl_value,exp_award_value,exp_award_natl_value,...,retain_percentile,ft_fac_value,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size,retention_class
0,100654,Alabama,4-year,Masters Colleges and Universities--larger prog...,NaN,-86.568502,4051,21.5,105331,66436,...,17.0,82.8,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0,0
1,100663,Alabama,4-year,Research Universities--very high research acti...,NaN,-86.809170,11502,21.5,136546,66436,...,70.0,92.4,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0,1
2,100690,Alabama,4-year,Baccalaureate Colleges--Arts & Sciences,NaN,-86.174010,322,22.5,58414,101725,...,2.0,67.2,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0,0
3,100706,Alabama,4-year,Research Universities--very high research acti...,NaN,-86.638420,5696,21.5,64418,66436,...,72.0,65.5,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0,1
4,100724,Alabama,4-year,Masters Colleges and Universities--larger prog...,NaN,-86.295677,5356,21.5,132407,66436,...,15.0,67.0,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0,0


In [24]:
# ensuring the percentile is accurate??
aid_percentile = list(college_completion.select_dtypes('integer'))

cc_aid = pd.get_dummies(college_completion, columns = aid_percentile) 

cc_aid.boxplot(column= 'aid_percentile', vert= False, grid=False)
cc_aid.aid_percentile.describe()

count    3797.000000
mean       49.991046
std        28.922664
min         0.000000
25%        25.000000
50%        50.000000
75%        75.000000
max       100.000000
Name: aid_percentile, dtype: float64

In [25]:
X = college_completion[['aid_percentile']]
nan_countX = X.isna().sum()
Y = college_completion[['retain_percentile']]
nan_countY = Y.isna().sum()


college_completion_clean = college_completion.dropna(subset=['aid_percentile', 'retain_percentile'])

print(nan_countX)
print(nan_countY)

X_clean = college_completion_clean[['aid_percentile']]
Y_clean = college_completion_clean[['retain_percentile']]

print(X_clean.isna().sum())
print(Y_clean.isna().sum())

X_train, X_test, Y_train, Y_test = train_test_split(X_clean, Y_clean, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

aid_percentile    1
dtype: int64
retain_percentile    263
dtype: int64
aid_percentile    0
dtype: int64
retain_percentile    0
dtype: int64
(2473, 1) (1061, 1) (2473, 1) (1061, 1)


In [26]:
from sklearn.metrics import precision_score
model = LinearRegression()
model.fit(X_train, Y_train) #train the regression model

#stores predicted values for target variable
y_test_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE, MSE, RMSE, and R²
mae = mean_absolute_error(Y_test, y_test_pred)
mse = mean_squared_error(Y_test, y_test_pred)
rmse = mse**0.5
r2 = r2_score(Y_test, y_test_pred)

# Print the metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 24.4323013268575
Mean Squared Error (MSE): 821.743256849874
Root Mean Squared Error (RMSE): 28.666064551135616
R-squared (R²): 0.017532196301410563


## Function

In [7]:
def aid_retention(x):
    X = college_completion[['aid_percentile']]
    nan_countX = X.isna().sum()
    Y = college_completion[['retain_percentile']]
    nan_countY = Y.isna().sum()
    college_completion_clean = college_completion.dropna(subset=['aid_percentile', 'retain_percentile'])
    X_clean = college_completion_clean[['aid_percentile']]
    Y_clean = college_completion_clean[['retain_percentile']]
    X_train, X_test, Y_train, Y_test = train_test_split(X_clean, Y_clean, test_size=0.3, random_state=42)
    model = LinearRegression()
    model.fit(X_train, Y_train) 

    y_test_pred = model.predict(X_test)

    mae = mean_absolute_error(Y_test, y_test_pred)
    mse = mean_squared_error(Y_test, y_test_pred)
    rmse = mse**0.5
    r2 = r2_score(Y_test, y_test_pred)

    return(mae, mse, rmse, r2)

aid_retention(college_completion)

(24.4323013268575, 821.743256849874, 28.666064551135616, 0.017532196301410563)

### Step 3:
As we can see from the mean absolute error, it's very far from zero meaning that the model is performing very poorly. This means that the model isn't effectively predicting the retention rate from the aid percentile that the school is in. In order to become more effective, this model should utilize other variables to try and more accurately predict the retention rate (create a better "line of best fit")